# Monitoring Models

> [!NOTE] Ensure to utilize Python 3.10 SDK V2 for this demonstration.

Initialize the variables with your endpoint name, deployment name, and API key.

In [1]:
online_endpoint_name = "<your_endpoint_name>"
deployment = "<your_deployment_name>"

## Invoke the endpoint

Setup the workspace handle and the MLClient for managing resources in your Azure Machine Learning workspace.

In [2]:
# Handle to the workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import mlflow

ml_client = MLClient.from_config(
    DefaultAzureCredential()
)

Found the config file in: /config.json


Retrieve the MLflow tracking URI from the workspace to enable logging and monitoring of the model.

In [3]:
# Gather MLflow URI information from workspace
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

Import necessary entities for interacting with the managed online endpoint.

In [4]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)

## Use the Model

Make sure to capture the API key from the **Endpoint** '**Consume**' page. Replace it with the code below for api_key

<img title="Endpoint Details" src="Media/endpoint-details-page.png">

Obtain the necessary details about the online endpoint, such as the scoring URI, which will be used to send data for predictions.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

api_key = ml_client.online_endpoints.get_keys(online_endpoint_name).primary_key

# Display existing traffic details
print(f"Traffic Details: {endpoint.traffic}")

# Display the scoring URI
print(f"Scoring URI: {endpoint.scoring_uri}")

url = endpoint.scoring_uri

Utilize the custom function `invoke_endpoint` from the `utils` module to send a request to the endpoint and obtain predictions.

In [6]:
from utils.invoke import invoke_endpoint

# Invoke the endpoint to get predictions
# predicted_classes = invoke_endpoint(url, deployment, api_key)

## Load production data and run inference in Batches

Load the production data, and convert it to a Pandas DataFrame for easier manipulation.

In [7]:
import mltable

# Load the data asset
data_asset = ml_client.data.get("diabetes-mltable-production", label="latest")

# Load the table and convert to Pandas DataFrame
tbl = mltable.load(data_asset.path)
df = tbl.to_pandas_dataframe()
df.head()  # Display the first few rows of the DataFrame


,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1763065,7,93,86,16,49,45.327927,0.204736,24,False
1,1814475,1,98,62,13,58,29.712845,0.087436,34,True
2,1473525,1,70,86,11,50,40.657364,0.492743,22,False
3,1123081,3,135,70,19,27,50.915305,0.111574,21,False
4,1479105,1,139,94,7,15,34.524436,0.122165,26,False


Prepare the data by separating the features (X) and the target variable (y), then split the data into batches for batch processing.

In [8]:
import numpy as np

# Separate features and target variable
X, y = df[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']], df['Diabetic']

# Split the DataFrame into batches of size 5
batches = [batch.values.tolist() for batch in np.array_split(X, len(X) / 5)]


Invoke the endpoint with each batch of data to get predictions.

In [9]:
predictions = []

import json
import tempfile
import os
from tqdm import tqdm

for batch in tqdm(batches):
    payload = {"input_data": batch, "params": {}}
    predictions_batch = invoke_endpoint(url, deployment, api_key, payload)
    predictions.extend(predictions_batch)

100%|██████████| 900/900 [00:57<00:00, 15.78it/s]


Evaluate the model's performance on the production data using accuracy and F1 score metrics.

In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions, columns=["Predicted"])

# Evaluate model performance
accuracy = accuracy_score(y_true=y, y_pred=predictions_df)
f1 = f1_score(y_true=y, y_pred=predictions_df)

print(f'Accuracy: {accuracy:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.77
F1 Score: 0.73
